In [ ]:
# Install dependencies

In [1]:
!pip install scikit-surprise --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# !pip install kaggle --quite

In [2]:
!pip install Dataset dataset --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.51 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from surprise import SVD, NMF, KNNBasic, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

In [ ]:
# Import files

In [5]:
import pandas as pd
# use rating_complete first, then animelist
# Surprise expects : user, item, ratings

# anime_df = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Recommender_Systems/Kaggle files/anime.csv")
# anime_with_syn = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Recommender_Systems/Kaggle files/anime_with_synopsis.csv")
animelist_df = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Recomender_Systems/Kaggle files/animelist.csv")
ratings_df = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Recomender_Systems/Kaggle files/rating_complete.csv", usecols=range(3), header=0, names=('user', 'item', 'rating'), nrows=500000)

# Personal dataset
df_reco = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Recomender_Systems/Kaggle files/reco_rates - Feuille 1.csv", header =1)

In [ ]:
# Using the Surprise library with the different
# The lower the RMSE / MSE the better

In [ ]:
from surprise import Dataset, Reader

# Prepare the dataset for Surprise
reader = Reader(rating_scale=(1, 10))

data = Dataset.load_from_df(ratings_df[['user', 'item', 'rating']], reader)

In [ ]:
# Use SVD algorithm
algo = SVD()

# Split the data into training and test set
trainset, testset = train_test_split(data, test_size=0.25)

# Train the algorithm using the trainset
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Compute the Root Mean Squared Error and Mean Squared Error
accuracy.rmse(predictions)
accuracy.mse(predictions)

RMSE: 1.2331
MSE: 1.5206


1.5206381921896646

In [ ]:
# Use the KNNBasic algorithm
algo = KNNBasic()

# Train the algorithm on the trainset
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Compute the Root Mean Squared Error and Mean Squared Error
accuracy.rmse(predictions)
accuracy.mse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3598
MSE: 1.8491


1.8491366018012765

In [ ]:
# Use the NMF algorithm
algo = NMF()

# Train the algorithm on the trainset
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Compute the Root Mean Squared Error and Mean Squared Error
accuracy.rmse(predictions)
accuracy.mse(predictions)

RMSE: 2.1210
MSE: 4.4985


4.498501411313473

In [ ]:
# Cross Validation and Grid Search

In [ ]:
# CROSS VALIDATION

# Parameter grid
param_grid = {
    'n_epochs': [5, 10, 20, 30],
    'lr_all': [0.002, 0.003, 0.005],
    'reg_all': [0.4, 0.6]
}

# Perform grid search
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mse'], cv=5)
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Lists the params with the best RMSE score
print(gs.best_params['rmse'])

1.3011919757945123
{'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
# Best MSE score
print("Best MSE score:", gs.best_score['mse'])

# Lists the params with the best MSE score
print("Best parameters for MSE:", gs.best_params['mse'])

Best MSE score: 1.6931094456899298
Best parameters for MSE: {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
# Use the NMF algorithm
algo = NMF()

# Perform cross validation
results = cross_validate(algo, data, measures=['rmse', 'mse'], cv=5, verbose=True)

Evaluating RMSE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.1245  2.1170  2.1153  2.1172  2.1288  2.1206  0.0052  
MSE (testset)     4.5137  4.4819  4.4743  4.4827  4.5320  4.4969  0.0221  
Fit time          13.71   14.98   14.90   16.69   15.99   15.25   1.02    
Test time         0.48    1.44    1.25    1.25    1.31    1.15    0.34    


In [ ]:
# List of algorithms
algos = [NMF(), SVD(), KNNBasic()]

# Iterate over all algorithms
for algo in algos:
    # Perform cross-validation
    results = cross_validate(algo, data, measures=['rmse', 'mse'], cv=5, verbose=True)

    # Display results for each algorithm
    print("Algorithm:", algo.__class__.__name__)
    print("Average RMSE:", sum(results['test_rmse']) / 5)
    print("Average MSE:", sum(results['test_mse']) / 5)
    print("\n")


Evaluating RMSE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.1359  2.1121  2.1235  2.1248  2.1373  2.1267  0.0092  
MSE (testset)     4.5621  4.4611  4.5092  4.5149  4.5680  4.5230  0.0391  
Fit time          14.40   15.41   14.71   15.13   14.83   14.90   0.35    
Test time         1.27    1.29    1.38    1.21    1.21    1.27    0.06    
Algorithm: NMF
Average RMSE: 2.1267254452877906
Average MSE: 4.523045641069475


Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2250  1.2318  1.2372  1.2313  1.2336  1.2318  0.0040  
MSE (testset)     1.5006  1.5174  1.5306  1.5161  1.5217  1.5173  0.0098  
Fit time          7.33    7.02    7.46    7.31    7.41    7.30    0.15    
Test time         1.42    1.35    1.36    1.39    1.41    1.39    0.03    
Algorithm: SVD
Average RMSE: 1.2317698917788529
Average MSE: 

In [6]:
import pandas as pd
from collections import Counter

def combine_df(df_reco:pd.DataFrame, df_comp:pd.DataFrame):
    """Combine the dataframe of our ratings and the dataframe of ratings, just load them"""
    df_reco_comp = pd.concat([df_reco.iloc[:,:2],df_reco[df_reco.iloc[:,2].name]], axis=1).rename(columns={df_reco.iloc[:,2].name:'rating'}) #
    df_reco_comp['user'] = df_reco.iloc[:,2].name #fill with name
    for col in df_reco.iloc[:,3:]:
        df_temp = pd.concat([df_reco.iloc[:,:2],df_reco[col]], axis=1).rename(columns={col:'rating'})
        df_temp['user'] = col #fill with name
        df_reco_comp = pd.concat([df_reco_comp, df_temp])
    df_reco_comp = df_reco_comp.rename(columns={'id':'item'})

    df_reco_comp = df_reco_comp.query('rating!=0') #filter animes that we dont know from our dataset
    df_reco_comp['rating'] = df_reco_comp.rating.apply(lambda x: 1 if x== -1 else 2) #set the rating scale to 1 or 2 for our dataset
    df_comp['rating'] = df_comp.rating.apply(lambda x: 1 if x<=5 else 2) #set the rating scale to 1 or 2 for the other dataset
    return pd.concat([df_comp,df_reco_comp.loc[:,['user','item','rating']]], ignore_index=True)

def compute_genres_stats(df:pd.DataFrame) -> dict:
    """Returns a dict with ratio for the genres in the Dataframe"""
    count_genre = {}
    for genre_list in df.Genres_list:
        for genre in genre_list:
            if genre not in count_genre.keys():
                count_genre[genre]=0
            count_genre[genre]+=1
    for genre, count in count_genre.items():
        count_genre[genre] = count/len(df)
    return count_genre

    # alt one-line version
    # return {k:v/len(df) for k,v in dict(Counter([genre for genre_list in df.Genres_list for genre in genre_list])).items()}

def genres_for_playlists(genre_personal:dict, genre_global:dict) -> dict:
    """Returns a dict ordered by the difference between ratios of the presences of genres in two dict.

    First dict should be the dict for which we want to make playlists for."""
    genres_diff = {}
    for genre, ratio in genre_personal.items():
        genres_diff[genre] = ratio-genre_global[genre]
    return dict(sorted(genres_diff.items(), key=lambda x: x[1], reverse=True))

In [7]:
from surprise import Dataset, Reader

df_full = combine_df(df_reco, ratings_df)
reader = Reader(rating_scale=(1,2))
data = Dataset.load_from_df(df_full, reader)

<ipython-input-6-18ed7f8c2fc6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reco_comp['rating'] = df_reco_comp.rating.apply(lambda x: 1 if x== -1 else 2) #set the rating scale to 1 or 2 for our dataset


In [ ]:
from surprise.model_selection import KFold
# Class that combine Gridsearch and Cross Validating function

class GridSearchCrossValidation:
    def __init__(self, algo_classes, param_grid, data, measures=['rmse', 'mse'], cv_strategy=KFold(n_splits=5), verbose=False):
        self.algo_classes = algo_classes
        self.param_grid = param_grid
        self.data = data
        self.measures = measures
        self.cv_strategy = cv_strategy
        self.verbose = verbose
        self.results = []

# Grid search method
    def grid_search(self):
        for algo_class in self.algo_classes:
            try:
                gs = GridSearchCV(algo_class, self.param_grid, measures=self.measures, cv=self.cv_strategy)
                gs.fit(self.data)

                result = {
                    'Algorithm': algo_class.__name__,
                    'Best Score': {measure: gs.best_score[measure] for measure in self.measures},
                    'Best Params': {measure: gs.best_params[measure] for measure in self.measures}
                }
                self.results.append(result)

                if self.verbose:
                    print(result)

            except Exception as e:
                print(f"Error processing {algo_class.__name__}: {str(e)}")

# Cross Validation method
    def cross_validate(self):
        for algo_class in self.algo_classes:
            try:
                algo = algo_class()
                results = cross_validate(algo, self.data, measures=self.measures, cv=self.cv_strategy, verbose=self.verbose)
                avg_results = {f'Average {measure}': sum(results[f'test_{measure}']) / len(results[f'test_{measure}']) for measure in self.measures}

                result = {
                    'Algorithm': algo_class.__name__,
                    'Cross-validation Results': avg_results
                }
                self.results.append(result)

                if self.verbose:
                    print(result)

            except Exception as e:
                print(f"Error processing {algo_class.__name__}: {str(e)}")

# To call both methods and run
    def run(self):
        self.grid_search()
        self.cross_validate()
        return pd.DataFrame(self.results)

# Global variables
algo_classes = [NMF, SVD, KNNBasic]
param_grid = {
    'n_epochs': [5, 10, 20, 30],
    'lr_all': [0.002, 0.003, 0.005],
    'reg_all': [0.4, 0.6]
}

# Instantiating and initialization of class
gs_cv = GridSearchCrossValidation(algo_classes, param_grid, data, verbose=True)
results_df = gs_cv.run()


Error processing NMF: __init__() got an unexpected keyword argument 'lr_all'
{'Algorithm': 'SVD', 'Best Score': {'rmse': 0.27255765762911865, 'mse': 0.07428966519869071}, 'Best Params': {'rmse': {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}, 'mse': {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}}}
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...

In [9]:
from surprise import Dataset, Reader, SVD, KNNBasic, NMF
from surprise.model_selection import GridSearchCV

# Function to combine Gridsearch and Cross validation using RMSE and MSE

def GridSearchCrossValidationMSE(data):
    # Define the algorithms and their unique parameters
    param_grid = {
        'SVD': {'n_epochs': [5, 10, 20, 30], 'lr_all': [0.002, 0.003, 0.005], 'reg_all': [0.02, 0.05]},
        'KNNBasic': {'k': [20, 30, 50], 'min_k': [1, 5]},
        'NMF': {'n_epochs': [5, 10, 20], 'n_factors': [15, 20, 25]}
    }

    # RMSE and MSE evaluation metrics
    metrics = ['rmse', 'mse']

    # Grid search with cross-validation for each algorithm
    for algo_name in param_grid:
        print(f"Evaluating {algo_name}...")
        gs = GridSearchCV(eval(algo_name), param_grid[algo_name], measures=metrics, cv=5)
        gs.fit(data)

        # Print best score and parameters
        print(f"Best RMSE for {algo_name}: {gs.best_score['rmse']}")
        print(f"Best MSE for {algo_name}: {gs.best_score['mse']}")
        print(f"Best Parameters for {algo_name}: {gs.best_params['rmse']}")

# Apply function
GridSearchCrossValidationMSE(data)

Evaluating SVD...
Best RMSE for SVD: 0.26909719499398504
Best MSE for SVD: 0.07241337779907109
Best Parameters for SVD: {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.05}
Evaluating KNNBasic...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Com

In [10]:
from surprise import Dataset, Reader, SVD, KNNBasic, NMF
from surprise.model_selection import cross_validate, GridSearchCV


# Function to combine Gridsearch and Cross validation using RMSE and MAE

def GridSearchCrossValidationMAE(data):
    # Dictionary of algorithms to be used
    algorithms = {
        'SVD': SVD,
        'KNNBasic': KNNBasic,
        'NMF': NMF
    }

    # Unique parameters for each algorithm
    param_grid = {
        'SVD': {'n_epochs': [5, 10, 20, 30], 'lr_all': [0.002, 0.003, 0.005]},
        'KNNBasic': {'k': [20, 30, 50], 'min_k': [1, 5]},
        'NMF': {'n_epochs': [5, 10, 20, 30], 'n_factors': [15, 20, 25]}
    }

    # Loop over the algorithms
    for name, algo in algorithms.items():
        gs = GridSearchCV(algo, param_grid[name], measures=['rmse', 'mae'], cv=5)
        gs.fit(data)

        # Print best score and parameters
        print(f"Best Score for {name}: {gs.best_score['rmse']}")
        print(f"Best Parameters for {name}: {gs.best_params['rmse']}")

# Apply function to dataset
GridSearchCrossValidationMAE(data)


Best Score for SVD: 0.2689598363983512
Best Parameters for SVD: {'n_epochs': 30, 'lr_all': 0.005}
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similari